In [1]:
# Imports and functions
from IPython.display import Image
import pandas
import csv
def displaycsv(filename):
    with open(filename) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            print row

# The Python FTA package in MASTODON

## Basic fault-tree analysis

This section demonstrates a basic fault-tree analysis in the `prautils` Python package in MASTODON. The demonstration starts with a fault tree described in a file called `events.csv`. This fault tree is the same as in the Saphire Technical manual, Vol 2. Only the minimal cutsets are calculated in this section. As shown in the code below, the execution can be done by initiating the `Quantification` class in the `FTA` module in the `prautils` package. The minimum inputs are the `name` (name of the problem), `logic` (csv file or a list containing the fault tree logic), and `basic_events` (csv file or list containing the basic events and probabilities associated with them. The fault tree described in this example is shown below. 

<img src="a/saphire_example_ft.png" alt="Drawing" style="width: 800px;"/>


An instance of the `Quantification` class called `saphire_fta` is first created. This class performs the fault-tree analysis and calculates the minimal cutsets automatically. These cutsets are then printed below.

In [2]:
from prautils import FTA
saphire_fta = FTA.Quantification(name='saphire_demo', logic='a/events.csv', basic_events='a/bas_events.csv')
print saphire_fta._Quantification__mcsets

[set([B2, B1]), set([B4, B1]), set([B1, B3, B5]), set([B5, B2, B3]), set([B4, B3, B5])]


## Quantifying the minimal cutsets for point estimate probabilities

The probabilities of the basic events are used to quantify each minimal cutset, namely, calculate their probability of occurrence. The sum of the probabilities of these minimal cutsets then provide the top event probability. The probabilities of the minimal cutsets calculated above and the top event probability for this fault tree are shown below. 

In [3]:
print pandas.read_csv(saphire_fta._Quantification__name+"_results/cutsets.csv")

       Cut Sets  Prob/Freq      IM (%)
0         Total   0.000705  100.000000
1      [B2, B1]   0.000200   28.374676
2      [B4, B1]   0.000400   56.749352
3  [B1, B3, B5]   0.000015    2.128101
4  [B5, B2, B3]   0.000030    4.256201
5  [B4, B3, B5]   0.000060    8.512403


Quantification of the top event probability is shown below. The results are presented for three types of top event probability calculations:
 - Upper bound
 - Rare event
 - Min max

Note that, due to the contents of the `bas_event.csv` file because this is a point estimate analysis, and therefore the probabilities, mean, median, etc. are the same for this problem. This is because the `bas_events.csv` file provides point estimate probabilities for the basic events. If a risk distribution is provided in the basic events file, a Monte Carlo simulation is performed and a distribution of the top event risk is calculated. This will be demonstrated later.

In [4]:
print pandas.read_csv(saphire_fta._Quantification__name+"_results/top_event.csv")

  Quantification Method  Prob/Freq      Mean    Median       5th      95th  \
0       MCS Upper Bound   0.000705  0.000705  0.000705  0.000705  0.000705   
1            Rare Event   0.000705  0.000705  0.000705  0.000705  0.000705   
2               Min Max   0.000694  0.000694  0.000694  0.000694  0.000694   

    SD  
0  0.0  
1  0.0  
2  0.0  


## Seismic PRA using MASTODON

MASTODON can also perform seismic PRA by propagating the component fragilities through the fault trees. The inputs for seismic PRA include the hazard curve, provided as a csv file, range of the intensity measure in the hazard, number of bins, and fragilities of the components in the basic events file. The fragilities in the basic events file should be expressed as lognormal distributions, indicated by the `LNORM` tag in the basic events file. The seismic risk can be calculated using the following ways. MASTODON can calculate the top event probability (or the system risk) using both procedures. 
 - Propagating fragilities
     1. The hazard curve is split into the number of bins input by the user. 
     2. For each bin, a probability of failure of the component is computed using the fragility curve.
     3. These probabilities are propagated through the fault tree and this computation is repeated for each bin. This results in a set of top event probabilities corresponding to the number of bins in the hazard curve.
     4. A lognormal distribution is fit into the top event probabilities vs. intensity measures (usually PGA) to calculate a system fragility curve. 
     5. The system fragility curve is then convolved with the hazard curve to calculate the risk.
 - Propagating risk
     1. The fragility curves of each component are first convolved with the hazard curve to calculate the total component risk of failure, given the seismic hazard. 
     2. The risks of failure of the components are propagated through the fault tree as point estimates to calculate the final risk. 

Only the second method can currently be A fresh PRA is performed below for the same fault tree, now assuming that all initiating events are seismic induced failures. Method 2, above is used to calculate the top event risk presented below. 

In [7]:
seismic_fta = FTA.Quantification(name='seismic_demo', 
                                         logic='b/seismic_logic.csv', 
                                         basic_events='b/seismic_bas_events_ln.csv', 
                                         analysis='Fragility', 
                                         hazard='b/seismic_hazard.csv',
                                         IM=[0.1, 2],
                                         nbins=12)
print seismic_fta._Quantification__mcsets
print pandas.read_csv(seismic_fta._Quantification__name+"_results/cutsets.csv")
print pandas.read_csv(seismic_fta._Quantification__name+"_results/top_event.csv")

TOPPPPP FRSGGGGGGG!! [set([SPumpF]), set([SDistPanelF]), set([SBlockWallF]), set([SBatF, SSWGearF])]
TOPPPPP FRSGGGGGGG!! [0.0001703761580608597, 0.0038919883026651927, 0.02008220991335506, 0.059633296851492816, 0.129018459444084, 0.22566387752956207, 0.3398382717593824, 0.4593351289031176, 0.5733127355572759, 0.6742951804265046, 0.7585602202163305, 0.8255004290201328]
[set([SPumpF]), set([SDistPanelF]), set([SBlockWallF]), set([SBatF, SSWGearF])]
OrderedDict([('MCS Upper Bound', [[2.821672013042509e-06, 2.821672013042509e-06, 2.821672013042509e-06, 2.821672013042509e-06, 2.821672013042509e-06, 0.0]]), ('Rare Event', [[2.8216734236261906e-06, 2.8216734236261906e-06, 2.8216734236261906e-06, 2.8216734236261906e-06, 2.8216734236261906e-06, 0.0]]), ('Min Max', [[2.821672013122199e-06, 2.821672013122199e-06, 2.821672013122199e-06, 2.821672013122199e-06, 2.821672013122199e-06, 0.0]])])
            Cut Sets     Prob/Freq      IM (%)
0              Total  2.821672e-06  100.000000
1           [

The system fragility parameters calculated using method 1 is calculated below. These parameters are calculated 

In [6]:
print "Median:\t", seismic_fta._Quantification__ln[0]
print "Beta:\t", seismic_fta._Quantification__ln[1]

Median:	1.328905707114457
Beta:	0.4161228970675382


In [13]:
print seismic_fta._Quantification__top_cal['MCS Upper Bound']

[[2.821672013042509e-06, 2.821672013042509e-06, 2.821672013042509e-06, 2.821672013042509e-06, 2.821672013042509e-06, 0.0]]
